## Building a CNN  Classifier

*   **Convolutional layers**, which apply a specified number of convolution
    filters to the image. For each subregion, the layer performs a set of
    mathematical operations to produce a single value in the output feature map.
    Convolutional layers then typically apply a
    [ReLU activation function](https://en.wikipedia.org/wiki/Rectifier_\(neural_networks\)) to
    the output to introduce nonlinearities into the model.

*   **Pooling layers**, which
    [downsample the image data](https://en.wikipedia.org/wiki/Convolutional_neural_network#Pooling_layer)
    extracted by the convolutional layers to reduce the dimensionality of the
    feature map in order to decrease processing time. A commonly used pooling
    algorithm is max pooling, which extracts subregions of the feature map
    (e.g., 2x2-pixel tiles), keeps their maximum value, and discards all other
    values.

*   **Dense (fully connected) layers**, which perform classification on the
    features extracted by the convolutional layers and downsampled by the
    pooling layers. In a dense layer, every node in the layer is connected to
    every node in the preceding layer.

### Imports


In [0]:
!pip install tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard

import cv2
import itertools
import numpy as np
import datetime, os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [0]:
# Clear any logs from previous runs
#!rm -rf ./logs/ 

## Network definition

In [0]:
class MyNet:
    
    def __init__(self,
                 n_epochs, 
                 batch_size, 
                 learning_rate,
                 input_shape,
                 logdir=None,
                 deeper=False):
        
        
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.input_shape = input_shape
        if logdir is None:
            self.logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        else:
            self.logdir = os.path.join("logs", logdir)
        
        if deeper:
            self._build_deeper_model()
            
        else:
            self._build_model()

    def _build_model(self):
        
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='relu', name="fc1"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation='softmax', name="predictions")
            ])
        print(self.input_shape)
        print("Model build")
        
    def _build_deeper_model(self):
        # Start of modified section
        # Model similar to LeNet-5 (but it doesn't seem to perform very well)
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu'),
            tf.keras.layers.AveragePooling2D(),
            tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'),
            tf.keras.layers.AveragePooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=120, activation='relu'),
            tf.keras.layers.Dense(units=84, activation='relu'),
            tf.keras.layers.Dense(units=10, activation = 'softmax')
        ])
        print("Deeper model build")
        # End of modified section

            
    def train_model(self, x_train, y_train):
        
        x_train, y_train, x_val, y_val = self._split_validation_data(x_train, 
                                                                     y_train, 
                                                                     0.1)

        
        optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        
        self.model.compile(optimizer=optimizer,
                           loss='sparse_categorical_crossentropy',
                           metrics=['accuracy'])
        
        
        tensorboard_callback = tf.keras.callbacks.TensorBoard(self.logdir, 
                                                              histogram_freq=1,
                                                              write_images=True, 
                                                              write_grads=True)
                                                              
        # Keras API for training
        # Start of modified section
        self.model.fit(x_train, y_train,
                       batch_size=64,
                       epochs=10,
                       validation_data=(x_val, y_val))
        # End of modified section
    
    @staticmethod
    def _split_validation_data(x, y, validation_split):
        rand_indexes = np.random.permutation(x.shape[0])
        x = x[rand_indexes]
        y = y[rand_indexes]
        x_validation = x[:int(len(x) * validation_split)]
        y_validation = y[:int(len(x) * validation_split)]
        x_train = x[int(len(x) * validation_split):]
        y_train = y[int(len(x) * validation_split):]
        return x_train, y_train, x_validation, y_validation

### Some plot functions

In [0]:
class MyPlot():
    def __init__(self,
                 nrows, 
                 ncols, 
                 figsize):
        self.fig, self.axes = plt.subplots(nrows=nrows, 
                                           ncols=ncols, 
                                           figsize=figsize)

        
def my_histogram(ax, data, color, title=None, rwidth=None, log=True, bins=25, align='mid', density=None):
    ax.hist(data, color=color, log=log, bins=bins, edgecolor='black', linewidth=1.2, rwidth=rwidth, align=align, density=density);
    ax.set_title(title);

    
def plot_random_images(images,
                       means,
                       labels=[],
                       examples=16, 
                       fig_suptitle=None, 
                       figsize=(8,8), 
                       fpath=None, 
                       imgs_index=None):
    
    if imgs_index is None:
        imgs_index = np.random.choice(np.arange(len(images)), examples, replace=False)
    plot = MyPlot(int(examples/np.sqrt(examples)), int(examples/np.sqrt(examples)), figsize=figsize)
    plot.axes = plot.axes.ravel()

    for idx, _ in enumerate(plot.axes):
        X_norm = images[imgs_index[idx]] * 255
        X = np.zeros(X_norm.shape, dtype=np.uint8)
        if len(X.shape) >= 3:
            for i in range(X.shape[-1]):
                X[:, :, i] = X_norm[:, :, i] + means[i]
                plot.axes[idx].imshow(X)
        else:
            X = X_norm + means
            plot.axes[idx].imshow(X, cmap="gray")
        plot.axes[idx].axis('off')
        if len(labels) > 0:
            plot.axes[idx].set_title(labels[imgs_index[idx]], fontsize=16, color="white")
        plot.fig.suptitle(fig_suptitle, fontsize=16)
    if fpath:
        plot.fig.savefig(fpath)
        

def compute_confusion_matrix(y_true, y_false):
    cm = confusion_matrix(y_true, y_false)
    return cm


def plot_confusion_matrix(cm,
                          labels_categorical,
                          title=None,
                          normalize=False, 
                          figsize=(8, 8)):
    
    plot = MyPlot(1, 1 ,figsize=figsize)
    ax = plot.axes
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    else:
        cm = cm
    ax.imshow(cm, interpolation='nearest', cmap=plt.cm.get_cmap("Greys"))

    tick_marks = np.arange(len(labels_categorical))
    ax.set_xticks(tick_marks)
    ax.set_xticklabels(labels_categorical, fontsize=16, color="white")
    ax.set_yticks(tick_marks)
    ax.set_yticklabels(labels_categorical, fontsize=16, color="white")

    fmt = '.2f'
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if np.isnan(cm[i, j]):
            cm[i, j] = 0
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        ax.text(j, i, format(cm[i, j], fmt),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")

    ax.set_ylabel('True label', fontsize=16, color="white")
    ax.set_xlabel('Predicted label', fontsize=16, color="white")
    ax.xaxis.set_tick_params(rotation=45)

    if title:
        ax.set_title(title)

## Download the FashionMNIST dataset and preprocess it

In [0]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print("Train dataset shape:\n\t Images {}".format(x_train.shape))
print("\t Labels {}".format(y_train.shape))

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

x_train_mean = np.mean(x_train, axis=(0, 1, 2))
print("Mean: {:.3f}".format(x_train_mean))
x_train = (x_train - x_train_mean)/255.0
x_test = (x_test - x_train_mean)/255.0

categorical_labels = [class_names[label] for label in y_train]
plot_random_images(x_train, x_train_mean, labels=categorical_labels)

## Train the model

In [0]:
my_net = MyNet(n_epochs=20,
               batch_size=64, 
               learning_rate=0.001, # 0.01
               input_shape=(28, 28, 1),
               deeper=True, 
               logdir="fc_0-001")

In [0]:
my_net.train_model(np.expand_dims(x_train, axis=-1), y_train)
loss, acc = my_net.model.evaluate(np.expand_dims(x_test, axis=-1), y_test, verbose=False);
predictions = my_net.model.predict(np.expand_dims(x_test, axis=-1))
print("Test done!\n\tMean accuracy: {}\n\tLoss: {}".format(acc, loss))

cm = compute_confusion_matrix(y_test, np.argmax(predictions, axis=1))
plot_confusion_matrix(cm, class_names, normalize=True)

## Tensorboard

In [0]:
%tensorboard --logdir logs/fc_0-001

## Biological case study

### Load data from gdrive

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

root_path = "gdrive/My Drive/Datasets/BIOSTEC_2018"
x_train_path = os.path.join(root_path, "train", "images.npy")
y_train_path = os.path.join(root_path, "train", "labels.npy")
x_test_path = os.path.join(root_path, "test", "images.npy")
y_test_path = os.path.join(root_path, "test", "labels.npy")

x_train = np.load(x_train_path)
y_train_categorical = np.load(y_train_path)
x_test = np.load(x_test_path)
y_test_categorical = np.load(y_test_path)

print("Train dataset:\n\tImages: {}\n\tLabels: {}".format(x_train.shape, y_train_categorical.shape))
print("Test dataset:\n\tImages: {}\n\tLabels: {}".format(x_test.shape, y_test_categorical.shape))

# Resize images to speed-up training
x_train = np.asarray([cv2.resize(image, (64, 64), 
                                 interpolation = cv2.INTER_CUBIC) for image in x_train])
x_test = np.asarray([cv2.resize(image, (64, 64), 
                                interpolation = cv2.INTER_CUBIC) for image in x_test])

# Convert labels to int
conversions = dict()
conversions['AC'] = 0
conversions['H'] = 1
conversions['AD'] = 2
y_train = np.asarray([conversions[label] for label in y_train_categorical])
y_test = np.asarray([conversions[label] for label in y_test_categorical])

# Normalization
# Start of modified section
# I cannot do this part, I don't have the dataset
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
# End of modified section

plot_random_images(x_train, x_train_mean, labels=y_train_categorical)

# Start of modified section
my_net = MyNet(n_epochs=20,
               batch_size=64, 
               learning_rate=0.001, # 0.01
               input_shape=(64, 64, 1),
               deeper=True, 
               logdir="fc_0-002")
# End of modified section

my_net.train_model(x_train, y_train)

In [0]:
loss, acc = my_net.model.evaluate(x_test, y_test, verbose=False);
predictions = my_net.model.predict(x_test)
print("Test done!\n\tMean accuracy: {}\n\tLoss: {}".format(acc, loss))

cm = compute_confusion_matrix(y_test, np.argmax(predictions, axis=1))
categorical_labels = ['AC', 'H', 'AD']
plot_confusion_matrix(cm, categorical_labels, normalize=True, figsize=(4, 4))